In [ ]:
# Import stuff
from os.path import join
from nipype.pipeline.engine import Workflow, Node
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from pandas import DataFrame, Series

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI')

# Set study variables
analysis_home = '/Users/catcamacho/Box/LNCD_rewards_connectivity'
#analysis_home = '/Volumes/Zeus/Cat'
preproc_dir = analysis_home + '/subjs'
raw_dir = analysis_home + '/subjs'
#raw_dir = '/Volumes/Phillips/bars/APWF_bars/subjs'
preproc_dir = analysis_home + '/proc/preprocessing'
firstlevel_dir = analysis_home + '/proc/firstlevel'
secondlevel_dir = analysis_home + '/proc/secondlevel'
workflow_dir = analysis_home + '/workflows'

#pull subject info to iter over
#subject_info = DataFrame.from_csv(analysis_home + '/misc/subjs.csv')
#subjects_list = subject_info['SubjID'].tolist()
#timepoints = subject_info['Timepoint'].tolist()

subjects_list = ['10766']
timepoints = [1]

# Seed locations and seed list
seed_dir = analysis_home + '/seeds'
L_amyg = seed_dir + '/L_amyg_anatomical.nii'
R_amyg = seed_dir + '/R_amyg_anatomical.nii'

seeds = [L_amyg, R_amyg]

In [ ]:
# Data handling nodes
infosource = Node(IdentityInterface(fields=['subjid','timepoint']), 
                  name='infosource')
infosource.iterables = [('subjid', subjects_list),('timepoint', timepoints)]
infosource.synchronize = True

#grab timing files
time_template = {'timing':raw_dir + '/%s/%d_*/timing/*score_timing.txt'}
timegrabber = Node(DataGrabber(sort_filelist=True,
                               template = raw_dir + '/%s/%d_*/timing/*score_timing.txt',
                               field_template = time_template,
                               base_directory=raw_dir,
                               infields=['subjid','timepoint'], 
                               template_args={'timing':[['subjid','timepoint']]}), 
                   name='timegrabber')

# Grab niftis
template = {'struct':preproc_dir + '/preproc_anat/{subjid}_t{timepoint}/reoriented_anat.nii',
            'func': preproc_dir + '/preproc_func/{subjid}_t{timepoint}/func_filtered.nii'}
datasource = Node(SelectFiles(template), 
                  name = 'datasource')

#sink important data
substitutions = [('_subjid_', ''),
                 ('_timepoint_','_t')]
datasink = Node(DataSink(substitutions=substitutions, 
                         base_directory=firstlevel_dir,
                         container=firstlevel_dir), 
                name='datasink')

In [ ]:
# Extract timing for Beta Series Method 
def timing_bars_punishment(run_timing_list):
    from os.path import abspath
    from pandas import DataFrame,Series,read_table,concat
    run_length = 453
    run_timing_list = sorted(run_timing_list)

    dfs = [ read_table(i,sep=' ') for i in run_timing_list ]
    k=1
    for df in dfs:
        df.loc[:,'runNum'] = Series(k, index = df.index)
        df.loc[:,'time_hyp'] = (k-1)*run_length + df.loc[:,'time_hyp']
        k = k+1
    df_full = concat(dfs)
    df_full = df_full.sort(['runNum','time_hyp'], ascending=[1,1])
    df_responded = df_full[df_full.loc[:,'Count'] == 1]
    df_responded = df_responded[df_responded.loc[:,'catch']==0]

    df_punish = df_responded[df_responded.loc[:,'cond']=='punish']
    df_reward = df_responded[df_responded.loc[:,'cond']=='reward']
    df_neutral = df_responded[df_responded.loc[:,'cond']=='neutral']
    return(timing_files_list)

In [1]:
from glob import glob
from os.path import abspath
from pandas import DataFrame,Series,read_table,concat
from math import isnan

run_timing_list = glob("/Users/catcamacho/Box/LNCD_rewards_connectivity/subjs/10766/3_20130130/timing/*score_timing.txt")
run_timing_list = sorted(run_timing_list)

dfs = [ read_table(i,sep=' ') for i in run_timing_list ]
k=1
for df in dfs:
    df.loc[:,'runNum'] = Series(k, index = df.index)
    df.loc[:,'time_hyp'] = (k-1)*453 + df.loc[:,'time_hyp']
    k = k+1
df_full = concat(dfs)
df_full = df_full.sort(['runNum','time_hyp'], ascending=[1,1])
df_responded = df_full[df_full.loc[:,'Count'] == 1]
df_responded = df_responded[df_responded.loc[:,'catch']==0]

df_punish = df_responded[df_responded.loc[:,'cond']=='punish']
df_reward = df_responded[df_responded.loc[:,'cond']=='reward']
df_neutral = df_responded[df_responded.loc[:,'cond']=='neutral']
#print(df_reward)
print(df_punish)
print(df_neutral)


     trial    lat  Count  time_hyp    cond  stim  catch  runNum
5        2  373.0      1       6.0  punish   cue      0       1
6        2  373.0      1       7.5  punish  prep      0       1
7        2  373.0      1       9.0  punish   sac      0       1
52       8  361.0      1      78.0  punish   cue      0       1
53       8  361.0      1      79.5  punish  prep      0       1
54       8  361.0      1      81.0  punish   sac      0       1
56       9  429.0      1      84.0  punish   cue      0       1
57       9  429.0      1      85.5  punish  prep      0       1
58       9  429.0      1      87.0  punish   sac      0       1
84      14  337.0      1     126.0  punish   cue      0       1
85      14  337.0      1     127.5  punish  prep      0       1
86      14  337.0      1     129.0  punish   sac      0       1
99      17  385.0      1     148.5  punish   cue      0       1
100     17  385.0      1     150.0  punish  prep      0       1
101     17  385.0      1     151.5  puni

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [ ]:
def create_design_mat(timing_files_list, motion):
    

In [ ]:
# Set up design matrix

# Run GLM

# Grab output betas and correlate to ROI

# Smooth parameter estimates

# Register to MNI space

# Extract ROI beta series

# Extract beta connectivity


In [ ]:
# Connect the workflow
level1workflow = Workflow(name='level1workflow')
level1workflow.connect([(infosource, datasource,[('subjid','subjid')]),
                        (infosource, datasource,[('timepoint','timepoint')]),
                        (datasource, merge, [('func','in_files')]),
                        
                        (merge,datasink,[('merged_file','merged_runs')])
                       ])
level1workflow.base_dir = join(workflow_dir)
level1workflow.write_graph(graph2use='flat')
level1workflow.run('MultiProc', plugin_args={'n_procs': 1})